# WebScraping the WikiPedia page to DataFrame


In [52]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd


In [53]:

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).text 
soup = BeautifulSoup(html, 'html5lib')

In [54]:
# Get table
table = soup.find('table')
contents = []

# Getting all rows
for row in table.find_all('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(cell)

In [55]:
# Creating the dataframe
df = pd.DataFrame(contents)

# Changing some values to more comprehensive names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                     'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                     'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                     'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [56]:
df.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [44]:
df.shape

(103, 3)

# Using Geocoder package to create the dataframe

In [6]:
!pip install geopy
!conda install -c conda-forge geocoder --yes


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|

pyasn1-0.4.8         | 53 KB     | ##################################### | 100% 
locket-0.2.0         | 6 KB      | ##################################### | 100% 
matplotlib-3.4.2     | 7 KB      | ##################################### | 100% 
hdf5-1.10.6          | 3.1 MB    | ##################################### | 100% 
jupyter_client-6.1.1 | 79 KB     | ##################################### | 100% 
libzopfli-1.0.3      | 164 KB    | ##################################### | 100% 
nbclient-0.5.3       | 67 KB     | ##################################### | 100% 
xorg-libxau-1.0.9    | 13 KB     | ##################################### | 100% 
zfp-0.5.5            | 190 KB    | ##################################### | 100% 
keras-preprocessing- | 34 KB     | ##################################### | 100% 
beautifulsoup4-4.9.3 | 86 KB     | ##################################### | 100% 
tensorflow-base-1.14 | 87.6 MB   | ##################################### | 100% 
requests-oauthlib-1. | 21 KB

attrs-21.2.0         | 44 KB     | ##################################### | 100% 
regex-2021.7.6       | 371 KB    | ##################################### | 100% 
sympy-1.8            | 11.1 MB   | ##################################### | 100% 
werkzeug-2.0.1       | 219 KB    | ##################################### | 100% 
google-auth-1.32.1   | 78 KB     | ##################################### | 100% 
blosc-1.21.0         | 841 KB    | ##################################### | 100% 
yarl-1.6.3           | 142 KB    | ##################################### | 100% 
ninja-1.10.2         | 2.4 MB    | ##################################### | 100% 
jupyter_core-4.7.1   | 72 KB     | ##################################### | 100% 
libprotobuf-3.17.2   | 2.5 MB    | ##################################### | 100% 
h5py-3.3.0           | 1.3 MB    | ##################################### | 100% 
protobuf-3.17.2      | 347 KB    | ##################################### | 100% 
zeromq-4.3.4         | 352 K

In [59]:
import geocoder # import geocoder
latitude=[]
longitude=[]
for code in df['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    #print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])
    
column_names = ['Postal Code','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Postal Code,Borough,Neighborhood,Latitude,Longitude


In [68]:
neighborhoods['Postal Code'] = df['PostalCode']
neighborhoods['Borough'] = df['Borough']
neighborhoods['Neighborhood'] = df['Neighborhood']
neighborhoods['Latitude'] = latitude
neighborhoods['Longitude'] = longitude
neighborhoods

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto Business,Enclave of M4L,43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


In [69]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


# Creating map of Toronto with Neighbourhoods superimposed on top

In [71]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  fol

In [73]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto
